In [ ]:
# How to easily play with the mypy type checker
# uv run mypy -c 'x = [1, 2]; print(x())'

In [ ]:
# why to use typing?

def mystery_function():
    return "I could be anything!"

# Use Any if you don't know the type of something or it's too
# dynamic to write a type for
x = mystery_function()
# Mypy will let you do anything with x!
x.whatever() * x["you"] + x("want") - any(x) and all(x) is super  # no errors

In [ ]:
# Use a "type: ignore" comment to suppress errors on a given line,
# when your code confuses mypy or runs into an outright bug in mypy.
# Good practice is to add a comment explaining the issue.

x = confusing_function()  # type: ignore  # confusing_function won't return None here because ...

In [ ]:
import typing # -> documentation
import collections.abc

# Small notes:

# very coupled with MyPy, a static type checker for Python - it is, de facto, standard
# however astral is watching the scene - https://docs.astral.sh/ty/

# based on work that Jukka Lehtosalo had done on his Ph.D. project—mypy.
# Python supports the concept of gradual typing. This means that you can gradually introduce types into your code. Code without type hints will be ignored by the static type checker.
# Type hints introduce a slight penalty in startup time. If you need to use the typing module the import time may be significant, especially in short scripts.
# https://realpython.com/python-type-checking
# https://mypy.readthedocs.io/en/stable/builtin_types.html

# Agenda:

# Basic overview
# reveal_type & cast
# TypeVar & ClassVar
# Stub files
# Protocol
# Python 3.11< & Python 3.12+
# Covariant & Contravariant & Invariant
# ParamSpec - PEP612 - https://sobolevn.me/2021/12/paramspec-guide
# Litelars & TypedDict & Enum

# Basic overview

In [ ]:
# Credit to Daria Obrusnikova 
# pre-commit config example:
  - repo: https://github.com/pre-commit/mirrors-mypy
    rev: v1.14.1
    hooks:
      # run mypy
      - id: mypy
        args: [ --no-strict-optional, --ignore-missing-imports ]
        additional_dependencies: [ types-python-dateutil, types-PyYAML ]
        files: "^(rag_service|tests)/"

# Github action example:
      - name: Install mypy
        run: pip install mypy

      - name: Run mypy
        run: mypy --no-strict-optional --ignore-missing-imports --install-types --non-interactive ./rag_service

In [ ]:
from typing import Any, Union, Optional, Tuple, Callable, Sequence # some of them can be imported from collections.abc in 3.10+

my_variable: Union[int, float] = 1. # int or float

my_variable: Optional[int, float] = 1. # Union[Union[int, float], None] -> Union[int, float, None]
my_variable: int | float | None = 1. # Python 3.10+ syntax

my_variable: Tuple[int, float] = (1, 2.) # order is important
my_variable: Tuple[float, int] = (1, 2.) # order is important
my_variable: Tuple[int, ...] = (1, 2, 3, 4) # any number of ints
my_variable: Sequence[Union[int, float]] = [1, 2., 3, 4.] # list or tuple of int or float
my_variable: list[int | str] = [3, 5, "test", "fun"]  # Python 3.10+


my_variable: Callable[[int, float], int] = lambda x, y: x + y # function that takes int and float and returns int

In [ ]:
# You don't need to initialize a variable to annotate it
# Doing so can be useful in conditional branches
child: bool
if age < 18:
    child = True
else:
    child = False


In [ ]:
# As the documentations says, if you are on Python 3.9+, you should most likely never use typing.Sequence 
# due to its deprecation. Since the introduction of generic alias types in 3.9 the collections.abc classes 
# all support subscripting and should be recognized correctly by static type checkers of all flavors.
from collections.abc import Mapping, MutableMapping, Sequence, Iterable

# Mapping describes a dict-like object (with "__getitem__") that we won't
# mutate, and MutableMapping one (with "__setitem__") that we might
def f(my_mapping: Mapping[int, str]) -> list[int]:
    my_mapping[5] = 'maybe'  # mypy will complain about this line...
    return list(my_mapping.keys())

f({3: 'yes', 4: 'no'})

def f(my_mapping: MutableMapping[int, str]) -> set[str]:
    my_mapping[5] = 'maybe'  # ...but mypy is OK with this.
    return set(my_mapping.values())

f({3: 'yes', 4: 'no'})


In [ ]:

# However, if you add the following special import:
from __future__ import annotations
# It will work at runtime and type checking will succeed as long as there
# is a class of that name later on in the file
def f(foo: A) -> int:  # Ok
    return 42

# Another option is to just put the type in quotes
def f(foo: 'A') -> int:  # Also ok
    ...

class A:
    # This can also come up if you need to reference a class in a type
    # annotation inside the definition of that class
    @classmethod
    def create(cls) -> A:
        return cls()

In [ ]:
from typing import Sequence, TypeVar

T = TypeVar("T")  # Declare type variable

def take_first(seq: Sequence[T]) -> T: # a generic function
    return seq[0]

accumulator = 0 # type: int

accumulator += take_first([1, 2, 3])   # Safe, T deduced to be int
accumulator += take_first((2.7, 3.5))  # Unsafe

In [ ]:
import random
from typing import Sequence, TypeVar

Choosable = TypeVar("Choosable")

def choose(items: Sequence[Choosable]) -> Choosable:
    return random.choice(items)

names = ["Guido", "Jukka", "Ivan"]
reveal_type(names)

name = choose(names)

# trying to find most speficif type
reveal_type(name)
reveal_type(choose(["Guido", "Jukka", "Ivan"]))
reveal_type(choose([1, 2, 3]))
reveal_type(choose([True, 42, 3.14]))
reveal_type(choose(["Python", 3, 7]))

# reveal_type & cast

In [ ]:
from typing import cast, reveal_type

# "cast" is a helper function that lets you override the inferred
# type of an expression. It's only for mypy -- there's no runtime check.

a = [4]
b = cast(list[int], a)  # Passes fine
c = cast(list[str], a)  # Passes fine despite being a lie (no runtime check)
reveal_type(c)  # Revealed type is "builtins.list[builtins.str]"
print(c)  # Still prints [4] ... the object is not changed or casted at runtime

# TypeVar & ClassVar

In [ ]:
from typing import TypeVar, Iterable, Tuple, List, reveal_type

T = TypeVar('T', int, float) # int or float
Vector = Iterable[Tuple[T, T]] # alias for complex types

def inproduct(v: Vector[T]) -> List[T]:
    return [x*y for x, y in v]

print(inproduct.__annotations__)
result = inproduct([(1, 2.3), (2, 3), (3, 4)])
print(type(result))
reveal_type(result)
print(result)

In [ ]:
from typing import Callable, TypeVar, Optional, reveal_type

def foo(f: Callable[[int], int]) -> Optional[int]:
    return f(1)

In [ ]:
from typing import ClassVar


# You can use the ClassVar annotation to declare a class variable
class Car:
    seats: ClassVar[int] = 4
    passengers: ClassVar[list[str]]

# Stub files

In [ ]:
# rename example_module_temp.pyi to example_module.pyi
from stub_example.example_module import concatenate_obj

a = concatenate_obj("1", "1.0")
print(a)

from stub_example.example_module import DataProcessor

data_processor = DataProcessor(list())

Typeshed is a Github repository that contains type hints for the Python standard library, as well as many third-party packages. Typeshed comes included with mypy so if you are using a package that already has type hints defined in Typeshed, the type checking will just work.

https://github.com/python/typeshed/tree/main

For example:
https://github.com/python/typeshed/tree/main/stubs/requests/requests

In [ ]:
# uv pip install types-requests
# uv pip uninstall types-requests

import requests
requests.get("https://httpbin.org/get")
"""
def get(url, params=None, **kwargs):
"""
requests.get(2)

In [ ]:
stubgen -p my_library
# MYPYPATH="$MYPYPATH:./out" mypy playground.py
# See details: https://mypy.readthedocs.io/en/stable/stubs.html#creating-a-stub

"""
# Variables with annotations do not need to be assigned a value.
# So by convention, we omit them in the stub file.
x: int

# Function bodies cannot be completely removed. By convention,
# we replace them with `...` instead of the `pass` statement.
def func_1(code: str) -> int: ...

# We can do the same with default arguments.
def func_2(a: int, b: int = ...) -> int: ...
"""

# Protocol

In [ ]:
# In a nominal system, comparisons between types are based on names and declarations. 
# The Python type system is mostly nominal, where an int can be used in place of a float because of their subtype relationship.
# Need to be "subclass" -> more like interface

# In a structural system, comparisons between types are based on structure. 
# You could define a structural type Sized that includes all instances that define .__len__(), irrespective of their nominal type.
# Related to duck typing -> more like protocols

In [ ]:
# Predefined Protocols: The typing module includes several predefined protocols, 
# such as typing.Sized, Container, Iterable, Awaitable, and ContextManager.

from typing import Protocol

class Sized(Protocol):
    def __len__(self) -> int: ...

def length(obj: Sized) -> int:
    return obj.__len__()

length("Hello")
length([1, 2, 3])

In [ ]:
from typing import Protocol, TypeVar, Generic

T = TypeVar('T')

class ReadWritable(Protocol[T]):
    def read(self) -> T: ...
    def write(self, data: T) -> None: ...

class FileHandler(Generic[T]):
    def __init__(self, data: T) -> None:
        self._data = data
    
    def read(self) -> T:
        return self._data
    
    def write(self, data: T) -> None:
        self._data = data

# Much cleaner and universally supported
def process_file(handler: ReadWritable[str]) -> None:
    content = handler.read()
    handler.write(content.upper())

# Usage example
file_handler = FileHandler("hello world")
process_file(file_handler)

In [ ]:
# can be run but mypy will complain

from datetime import date

class Animal:
    def __init__(self, name: str, birthday: date) -> None:
        self.name = name
        self.birthday = birthday

    @classmethod
    def newborn(cls, name: str) -> "Animal":
        return cls(name, date.today())

    def twin(self, name: str) -> "Animal":
        cls = self.__class__
        return cls(name, self.birthday)

class Dog(Animal):
    def bark(self) -> None:
        print(f"{self.name} says woof!")

fido = Dog.newborn("Fido")
pluto = fido.twin("Pluto")
fido.bark()
pluto.bark()

In [ ]:
from datetime import date
from typing import Type, TypeVar

# We specify that Animal is an upper bound for TAnimal. 
# Specifying bound means that TAnimal will only be Animal or one of its subclasses. 
# This is needed to properly restrict the types that are allowed.
TAnimal = TypeVar("TAnimal", bound="Animal")

class Animal:
    def __init__(self, name: str, birthday: date) -> None:
        self.name = name
        self.birthday = birthday

    @classmethod
    def newborn(cls: Type[TAnimal], name: str) -> TAnimal:
        return cls(name, date.today())

    def twin(self: TAnimal, name: str) -> TAnimal:
        cls = self.__class__
        return cls(name, self.birthday)

class Dog(Animal):
    def bark(self) -> None:
        print(f"{self.name} says woof!")

fido = Dog.newborn("Fido")
pluto = fido.twin("Pluto")
fido.bark()
pluto.bark()


In [ ]:
# how to suppress mypy errors

# mypy --ignore-missing-imports playground.py
# or # type: ignore
# or mypy.ini

import numpy as np

def print_cosine(x: np.ndarray) -> None:
    with np.printoptions(precision=3, suppress=True):
        print(np.cos(x))

x = np.linspace(0, 2 * np.pi, 9)
print_cosine(x)

# Pydantic

In [ ]:
from datetime import datetime
from typing import Optional
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: list[int] = []

external_data = {'id': '123', 'signup_ts': '2017-06-01 12:22', 'friends': [1, '2', b'3']}
user = User(**external_data)
print(user)

In [ ]:
# see https://mypy.readthedocs.io/en/latest/config_file.html#config-file
# mypy.ini (or in pyproject.toml)
# Can be per-module or global
[mypy]
plugins = pydantic.mypy

# Python 3.11< & Python 3.12+

In [ ]:
# New in Python 3.12
# else 
# from typing import TypeVar
# T = TypeVar("T")

class Box[T]:
    def __init__(self, content: T) -> None:
        self.content = content

Box(1)       # OK, inferred type is Box[int]
Box[int](1)  # Also OK

# error: Argument 1 to "Box" has incompatible type "str"; expected "int"
Box[int]('some string')

In [ ]:
from collections.abc import Sequence

def first[T](seq: Sequence[T]) -> T:
    return seq[0]

# or older version

from collections.abc import Sequence

from typing import TypeVar
T = TypeVar("T")

def first(seq: Sequence[T]) -> T:
    return seq[0]

In [ ]:
# same as 
# T = TypeVar('T', bound='Shape')

class Shape:
    def set_scale[T: Shape](self: T, scale: float) -> T:
        self.scale = scale
        return self

class Circle(Shape):
    def set_radius(self, r: float) -> 'Circle':
        self.radius = r
        return self

class Square(Shape):
    def set_width(self, w: float) -> 'Square':
        self.width = w
        return self

circle: Circle = Circle().set_scale(0.5).set_radius(2.7)
square: Square = Square().set_scale(0.5).set_width(3.2)

# Covariant & Contravariant & Invariant

Assuming that we have a pair of types A and B, and B is a subtype of A, these are defined as follows:

* A generic class MyCovGen[T] is called covariant in type variable T if MyCovGen[B] is always a subtype of MyCovGen[A].
    * From parent to child -> allow more specific - Sequence

* A generic class MyContraGen[T] is called contravariant in type variable T if MyContraGen[A] is always a subtype of MyContraGen[B].
    * From child to parent -> allow more generic - Callable

* A generic class MyInvGen[T] is called invariant in T if neither of the above is true.
    * Do not allow different type at all - List

Covariance means that if `Dog` is a subtype of `Animal`, then `Container[Dog]` should also be a subtype of `Container[Animal]`. However, allowing a covariant type variable as a parameter would break this rule. For instance:

```python
    container: Container[Animal] = Container[Dog]()
    container.add(Cat()) # This would be invalid but allowed if covariance were enforced
```

In [ ]:
# We'll use these classes in the examples below
class Shape: ...
class Triangle(Shape): ...
class Square(Shape): ...

def count_lines(shapes: Sequence[Shape]) -> int:
    return sum(shape.num_sides for shape in shapes)

triangles: Sequence[Triangle]
count_lines(triangles)  # OK

In [ ]:
"""
cost_of_paint_required needs a callable that can calculate the area of a triangle.
If we give it a callable that can calculate the area of an arbitrary shape (not just triangles),
everything still works.

"""
from typing import Callable

class Shape: ...
class Triangle(Shape): ...
class Square(Shape): ...

def cost_of_paint_required(
    triangle: Triangle,
    area_calculator: Callable[[Triangle], float]
) -> float:
    return area_calculator(triangle) * DOLLAR_PER_SQ_FT

# This straightforwardly works
def area_of_triangle(triangle: Triangle) -> float: ...
cost_of_paint_required(triangle, area_of_triangle)  # OK

# But this works as well!
def area_of_any_shape(shape: Shape) -> float: ...
cost_of_paint_required(triangle, area_of_any_shape)  # OK


In [ ]:
class Shape: ...
class Triangle(Shape): ...
class Square(Shape): ...

class Circle(Shape):
    # The rotate method is only defined on Circle, not on Shape
    def rotate(self): ...

def add_one(things: list[Shape]) -> None:
    things.append(Shape())

my_circles: list[Circle] = []
add_one(my_circles)     # This may appear safe, but...
my_circles[-1].rotate()  # ...this will fail, since my_circles[0] is now a Shape, not a Circle

In [ ]:
from typing import Generic, TypeVar

class Object: ...
class Shape(Object): ...
class Triangle(Shape): ...
class Square(Shape): ...

T = TypeVar('T', covariant=False, contravariant=False)  # invariant
# T = TypeVar('T', covariant=True, contravariant=False)  # covariant
# T = TypeVar('T', covariant=False, contravariant=True)  # contravariant

class Box(Generic[T]):
    def __init__(self, content: T) -> None:
        self._content = content

    # def get_content(self) -> T: # Cannot use a contravariant type variable as return
    #     return self._content

def look_into_obj(box: Box[Object]): ...

def look_into_shape(box: Box[Shape]): ...

def look_into_triangle(box: Box[Triangle]): ...


# obj = Object()
obj = Shape()
# obj = Triangle()

my_box = Box(obj)

look_into_obj(my_box)
look_into_shape(my_box)
look_into_triangle(my_box) 

In [ ]:
from typing import Generic, TypeVar

class Object: ...
class Shape(Object): ...
class Triangle(Shape): ...
class Square(Shape): ...

# Define TypeVars for different variance examples
T_invariant = TypeVar('T_invariant')  # invariant (default)
T_covariant = TypeVar('T_covariant', covariant=True)  # covariant
T_contravariant = TypeVar('T_contravariant', contravariant=True)  # contravariant

# Invariant container - only accepts exact type match
class InvariantContainer(Generic[T_invariant]):
    def __init__(self, item: T_invariant) -> None:
        self.item = item
    
    def get(self) -> T_invariant:
        return self.item
    
    def set(self, item: T_invariant) -> None:
        self.item = item

# Covariant container - can read subtype as supertype (Producer)
class CovariantContainer(Generic[T_covariant]):
    def __init__(self, item: T_covariant) -> None:
        self.item = item
    
    def get(self) -> T_covariant:  # Only output, no input
        return self.item

# Contravariant container - can accept supertype as subtype (Consumer)
class ContravariantContainer(Generic[T_contravariant]):
    def __init__(self) -> None:
        pass
    
    def process(self, item: T_contravariant) -> None:  # Only input, no output
        print(f"Processing: {type(item).__name__}")

# Examples demonstrating variance
def demonstrate_variance():
    # Invariant - exact type match required
    triangle_invariant: InvariantContainer[Triangle] = InvariantContainer(Triangle())
    # shape_invariant: InvariantContainer[Shape] = triangle_invariant  # This would be an error
    
    # Covariant - Triangle container can be treated as Shape container (reading)
    triangle_covariant: CovariantContainer[Triangle] = CovariantContainer(Triangle())
    shape_covariant: CovariantContainer[Shape] = triangle_covariant  # OK - covariant
    
    # Contravariant - Shape processor can process Triangles (writing)
    shape_contravariant: ContravariantContainer[Shape] = ContravariantContainer()
    triangle_contravariant: ContravariantContainer[Triangle] = shape_contravariant  # OK - contravariant
    
    # Usage examples
    triangle_contravariant.process(Triangle())  # OK - can process Triangle
    # triangle_contravariant.process(Square())    # ERROR - Square is not a Triangle!
    
    # Better contravariant example:
    # A Shape processor can handle any Shape (including Triangle and Square)
    shape_processor: ContravariantContainer[Shape] = ContravariantContainer()
    shape_processor.process(Triangle())  # OK
    shape_processor.process(Square())    # OK
    shape_processor.process(Shape())     # OK
    
    # Because shape_processor can handle Shapes, it can also be used as a Triangle processor
    triangle_processor: ContravariantContainer[Triangle] = shape_processor  # OK - contravariant
    triangle_processor.process(Triangle())  # OK - Triangle is what it expects


# ParamSpec

In [ ]:
from collections.abc import Callable
from typing import Any, TypeVar, cast

F = TypeVar('F', bound=Callable[..., Any])

# A decorator that preserves the signature.
def printing_decorator(func: F) -> F:
    def wrapper(*args, **kwds):
        print("Calling", func)
        return func(*args, **kwds)
    return cast(F, wrapper)

@printing_decorator
def add_forty_two(value: int) -> int:
    return value + 42

a = add_forty_two(3)
reveal_type(a)      # Revealed type is "builtins.int"
add_forty_two('x')  # Argument 1 to "add_forty_two" has incompatible type "str"; expected "int"

In [ ]:
# python 3.12+
from collections.abc import Callable

def printing_decorator[**P, T](func: Callable[P, T]) -> Callable[P, T]:
    def wrapper(*args: P.args, **kwds: P.kwargs) -> T:
        print("Calling", func)
        return func(*args, **kwds)
    return wrapper

# Python 3.11 and earlier
from collections.abc import Callable
from typing import TypeVar
from typing_extensions import ParamSpec

P = ParamSpec('P')
T = TypeVar('T')

def printing_decorator(func: Callable[P, T]) -> Callable[P, T]:
    def wrapper(*args: P.args, **kwds: P.kwargs) -> T:
        print("Calling", func)
        return func(*args, **kwds)
    return wrapper

In [ ]:
from collections.abc import Callable
from typing import TypeVar
from typing_extensions import Concatenate, ParamSpec

P = ParamSpec('P')
T = TypeVar('T')

def printing_decorator(func: Callable[P, T]) -> Callable[Concatenate[str, P], str]:
    def wrapper(msg: str, /, *args: P.args, **kwds: P.kwargs) -> str:
        print("Calling", func, "with", msg)
        return str(func(*args, **kwds))
    return wrapper

@printing_decorator
def add_forty_two(value: int) -> int:
    return value + 42

a = add_forty_two('three', 3)

In [ ]:
from typing import Callable, TypeVar, Optional, reveal_type
from typing_extensions import ParamSpec  # or `typing` for `python>=3.10`

T = TypeVar('T')
P = ParamSpec('P')

def catch_exception(function):
    return function

# def catch_exception(function: Callable[..., T]) -> Callable[..., Optional[T]]:
#     return function

# def catch_exception(function: Callable[P, T]) -> Callable[P, Optional[T]]:
#     def decorator(*args: P.args, **kwargs: P.kwargs) -> Optional[T]:
#         try:
#             return function(*args, **kwargs)
#         except Exception:
#             return None
#     return decorator

@catch_exception
def div(arg: int) -> float:
    return arg / arg

reveal_type(div)

@catch_exception
def plus(arg: int, other: int) -> int:
    return arg + other

reveal_type(plus)

In [ ]:
from typing_extensions import ParamSpec, Concatenate 
from typing import Callable  # or `typing` for `python>=3.10`

P = ParamSpec('P')

def bar(x: int, *args: bool) -> int: ...

def add(x: Callable[P, int]) -> Callable[Concatenate[str, P], int]: ...

add(bar)  # (str, /, x: int, *args: bool) -> int

def remove(x: Callable[Concatenate[int, P], int]) -> Callable[P, int]: ...

remove(bar)  # (*args: bool) -> int

def transform(
    x: Callable[Concatenate[int, P], int]
) -> Callable[Concatenate[str, P], bool]: ...

transform(bar)  # (str, /, *args: bool) -> bool

reveal_type(add)
reveal_type(remove)
reveal_type(transform)

# Litelars

In [ ]:
from typing import overload, Union, Literal

PrimaryColors = Literal["red", "blue", "yellow"]
SecondaryColors = Literal["purple", "green", "orange"]
AllowedColors = Literal[PrimaryColors, SecondaryColors]

def paint(color: AllowedColors) -> None: ...

paint("red")        # Type checks!
paint("turquoise")  # Does not type check

In [ ]:
tup = ("foo", 3.4)

# Indexing with an int literal gives us the exact type for that index
reveal_type(tup[0])  # Revealed type is "str"

# But what if we want the index to be a variable? Normally mypy won't
# know exactly what the index is and so will return a less precise type:
int_index = 0
reveal_type(tup[int_index])  # Revealed type is "Union[str, float]"

# But if we use either Literal types or a Final int, we can gain back
# the precision we originally had:
lit_index: Literal[0] = 0
fin_index: Final = 0
reveal_type(tup[lit_index])  # Revealed type is "str"
reveal_type(tup[fin_index])  # Revealed type is "str"


# TypedDict & Enum

In [ ]:
from typing import TypedDict

# We can do the same thing with with TypedDict and str keys:
class MyDict(TypedDict):
    name: str
    main_id: int
    backup_id: int

d: MyDict = {"name": "Saanvi", "main_id": 111, "backup_id": 222}
name_key: Final = "name"
reveal_type(d[name_key])  # Revealed type is "str"

# You can also index using unions of literals
id_key: Literal["main_id", "backup_id"]
reveal_type(d[id_key])    # Revealed type is "int"

In [ ]:
from typing import Literal, TypedDict, Union

class NewJobEvent(TypedDict):
    tag: Literal["new-job"]
    job_name: str
    config_file_path: str

class CancelJobEvent(TypedDict):
    tag: Literal["cancel-job"]
    job_id: int

Event = Union[NewJobEvent, CancelJobEvent]

def process_event(event: Event) -> None:
    # Since we made sure both TypedDicts have a key named 'tag', it's
    # safe to do 'event["tag"]'. This expression normally has the type
    # Literal["new-job", "cancel-job"], but the check below will narrow
    # the type to either Literal["new-job"] or Literal["cancel-job"].
    #
    # This in turns narrows the type of 'event' to either NewJobEvent
    # or CancelJobEvent.
    if event["tag"] == "new-job":
        print(event["job_name"])
    else:
        print(event["job_id"])

process_event({"tag": "new-job", "job_name": "example", "config_file_path": "/tmp/config"})
process_event({"tag": "cancel-job", "job_id": 12345})

In [ ]:
from typing import Literal, NoReturn
from typing_extensions import assert_never

PossibleValues = Literal['one', 'two', 'three']

def validate(x: PossibleValues) -> bool:
    if x == 'one':
        return True
    elif x == 'two':
        return False
    # Error: Argument 1 to "assert_never" has incompatible type "Literal['three']";
    # expected "NoReturn"
    assert_never(x)

In [ ]:
from enum import Enum

class Direction(Enum):
    up = 'up'
    down = 'down'

reveal_type(Direction.up)  # Revealed type is "Literal[Direction.up]?"
reveal_type(Direction.down)  # Revealed type is "Literal[Direction.down]?"

# Any Enum class with values is implicitly final. This is what happens in CPython:
